 ## 1 - Install/Upgrade and import required libraries

In [ ]:
# Install gluon ts from master-branch to fix error in Negative Binomial Distribution
%pip install git+https://github.com/awslabs/gluon-ts.git
%pip install --upgrade pandas==1.0
%pip install --upgrade mxnet==1.6

In [ ]:
%matplotlib inline
import mxnet as mx
from mxnet import gluon
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import os
from tqdm.autonotebook import tqdm
from pathlib import Path

## 2 - Select prediction length, input path and validation
#### If validation is True, the last prediction_length time points of the training set will be used as the testing set, to perform hyper-parameter tuning.

In [ ]:
validation = True

prediction_length = 48

PATH="../input/rossmann-prepared-dataset"

## 3 - Load and display data

In [ ]:
df = pd.read_csv(f'{PATH}/rossmann_train.csv')
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 1000)
df.head(10)

## 4 - Feature engineering, data cleaning and analysis
### 4.1 - Convert categoricals to numerical
* Convert every categorical feature that we include in dynamic_features to a numerical value.
* Store category code value in a new feature called 'feature-name_cat'.

In [ ]:
# Select categorical variables
cat_vars = ['Store', 'DayOfWeek', 'Year', 'Month', 'Day', 'StateHoliday', 'CompetitionMonthsOpen',
    'Promo2Weeks', 'StoreType', 'Assortment', 'PromoInterval', 'CompetitionOpenSinceYear', 'Promo2SinceYear',
    'State', 'Week', 'Events', 'Promo_fw', 'Promo_bw', 'StateHoliday_fw', 'StateHoliday_bw',
    'SchoolHoliday_fw', 'SchoolHoliday_bw']

# Make sure they are all of type category
for v in cat_vars: df[v] = df[v].astype('category').cat.as_ordered()

# Add column to dataframe with value of 'category.code'
for v in cat_vars: 
    df[v+'_cat'] = df[v].cat.codes    



### 4.2 - Count number of time series and time points per serie
#### Since we have the daily sales of each store, each time series is identified by the store id.

In [ ]:
# Get number of time series
number_ts=len(df.Store.unique())
print('Number of time series:\n', number_ts)

# Get number of timepoints in each time serie
print('List of time series lengths:\n',df['Store'].value_counts())

Check how many time series there are with the maximum number of time points

In [ ]:
ts_length = max(df['Store'].value_counts())

countEqual = 0
countOver = 0
countUnder = 0

# Check how many time series have length equal to ts_length
for v in df['Store'].value_counts():
    if(v == ts_length):
        countEqual += 1
        
# Check how many time series have length over ts_length
for v in df['Store'].value_counts():
    if(v > ts_length):
        countOver += 1
        
# Check how many time series have length under ts_length
for v in df['Store'].value_counts():
    if(v < ts_length):
        countUnder += 1
        
countEqualPercentage = round(100*countEqual/number_ts,1)
countOverPercentage = round(100*countOver/number_ts,1)
countUnderPercentage = round(100*countUnder/number_ts,1)

print(f'Time series with {ts_length} time-points: {countEqual} ({countEqualPercentage}%)\nTime series with more than {ts_length} time-points: {countOver} ({countOverPercentage}%)\nTime series with less than {ts_length} time-points: {countUnder} ({countUnderPercentage}%)')



### 4.3 - Remove Time Series with missing values
#### We can see that out of our 1115 time series, 181 (16%) have missing values, some of them for over 150 time points, this can negatively impact our models ability to make accurate predictions, so we will remove them

In [ ]:
# Remove Stores with missing dates, sort by Date and Store

df = df.groupby('Store').filter(lambda x : len(x)==ts_length)
df = df.sort_values(['Date','Store'])
pd.set_option('display.max_columns', 90)

#remove unnamed leftover column from index
df.columns.str.match('Unnamed')
# array([ True, False, False, False])

df=df.loc[:, ~df.columns.str.match('Unnamed')]
# Display data
df.head(5)

## 5 - Prepare data
### 5.1 - Build target values (sales) list

In [ ]:
# Update number of ts
number_ts = len(df.Store.unique())

stack = df.iloc[0:number_ts].Sales
count = number_ts

for i in range(ts_length-1):
    curr_idx = count + number_ts
    stack = np.column_stack((stack, df.iloc[count:curr_idx].Sales))
    count += number_ts

test_target_values = stack.copy()
train_target_values = test_target_values[:,:-prediction_length]
    
# Validation

# Use the prediction_length days prior to the days used for 
# testing as the test set to perform hyper-parameter tuning

# training-set-length = ts_length - prediction_length*2
# testing-set-length = ts_length - prediction_length
if(validation):
    test_target_values = test_target_values[:,:-prediction_length]
    train_target_values = test_target_values[:,:-prediction_length]

# Ensure target values list has the correct shape (number_ts, ts_length)
# number_ts - number of time series | ts_length - time series length
print('Expected shape: (number_ts, ts_length)\n')
print(f'Number of time series: {number_ts}\nTime series length: {ts_length}\n')
print(f'Target testing values shape: {test_target_values.shape} \nTarget training values shape (excluding last {prediction_length} time steps): {train_target_values.shape}')

### 5.2 - Build dynamic features list
#### We will create several sets of dynamic features to assess their impact on the performance

In [ ]:
# Dynamic features - Open
dynamic_features_open = df.drop(
    ['Sales', 'Date', 'Store', 'DayOfWeek', 'Year', 'Month', 'Day', 'StateHoliday', 'CompetitionMonthsOpen',
     'Promo2Weeks', 'StoreType', 'Assortment', 'PromoInterval', 'CompetitionOpenSinceYear', 'Promo2SinceYear',
     'State', 'Week', 'Events', 'Store_cat', 'Assortment_cat', 'StoreType_cat', 'State_cat',
     'SchoolHoliday_fw','SchoolHoliday_bw', 'StateHoliday_fw', 'StateHoliday_bw', 'Promo_bw', 'Promo_fw', 
     'Max_TemperatureC', 'Mean_TemperatureC', 'Min_TemperatureC', 'Max_Humidity', 'Mean_Humidity', 'Min_Humidity', 
     'Max_Wind_SpeedKm_h', 'Mean_Wind_SpeedKm_h','CloudCover', 'Events_cat','StateHoliday', 'SchoolHoliday',
     'CompetitionDistance', 'trend', 'trend_DE', 'AfterStateHoliday', 'BeforeStateHoliday', 'Promo', 'DayOfWeek_cat', 'Year_cat', 'Month_cat',
     'Day_cat', 'StateHoliday_cat', 'CompetitionMonthsOpen_cat', 'Promo2Weeks_cat', 'PromoInterval_cat', 'CompetitionOpenSinceYear_cat', 'Promo2SinceYear',
     'Week_cat', 'Promo_fw_cat', 'Promo_bw_cat', 'StateHoliday_fw_cat', 'StateHoliday_bw_cat', 'SchoolHoliday_fw_cat','SchoolHoliday_bw_cat', 'Promo2SinceYear_cat'],
    axis=1
)

# Dynamic features - Open / Promotions
dynamic_features_open_promo = df.drop(
    ['Sales', 'Date', 'Store', 'DayOfWeek', 'Year', 'Month', 'Day', 'StateHoliday', 'CompetitionMonthsOpen',
     'Promo2Weeks', 'StoreType', 'Assortment', 'PromoInterval', 'CompetitionOpenSinceYear', 'Promo2SinceYear',
     'State', 'Week', 'Events', 'Store_cat', 'Assortment_cat', 'StoreType_cat', 'State_cat',
     'SchoolHoliday_fw','SchoolHoliday_bw', 'StateHoliday_fw', 'StateHoliday_bw', 'Promo_bw', 'Promo_fw', 
     'Max_TemperatureC', 'Mean_TemperatureC', 'Min_TemperatureC', 'Max_Humidity', 'Mean_Humidity', 'Min_Humidity', 
     'Max_Wind_SpeedKm_h', 'Mean_Wind_SpeedKm_h','CloudCover', 'Events_cat','StateHoliday', 'SchoolHoliday',
     'CompetitionDistance', 'trend', 'trend_DE', 'AfterStateHoliday', 'BeforeStateHoliday', 'DayOfWeek_cat', 'Year_cat', 'Month_cat',
     'Day_cat', 'StateHoliday_cat', 'CompetitionMonthsOpen_cat', 'CompetitionOpenSinceYear_cat',
     'Week_cat', 'StateHoliday_fw_cat', 'StateHoliday_bw_cat', 'SchoolHoliday_fw_cat','SchoolHoliday_bw_cat'],
    axis=1
)

# Dynamic features - Open / Promotions / Holidays
dynamic_features_open_promo_holidays = df.drop(
    ['Sales', 'Date', 'Store', 'DayOfWeek', 'Year', 'Month', 'Day', 'CompetitionMonthsOpen',
     'Promo2Weeks', 'StoreType', 'Assortment', 'PromoInterval', 'CompetitionOpenSinceYear', 'Promo2SinceYear',
     'State', 'Week', 'Events', 'Store_cat', 'Assortment_cat', 'StoreType_cat', 'State_cat',
     'SchoolHoliday_fw','SchoolHoliday_bw', 'StateHoliday_fw', 'StateHoliday_bw', 'Promo_bw', 'Promo_fw', 
     'Max_TemperatureC', 'Mean_TemperatureC', 'Min_TemperatureC', 'Max_Humidity', 'Mean_Humidity', 'Min_Humidity', 
     'Max_Wind_SpeedKm_h', 'Mean_Wind_SpeedKm_h','CloudCover', 'Events_cat',
     'CompetitionDistance', 'trend', 'trend_DE', 'DayOfWeek_cat', 'Year_cat', 'Month_cat',
     'Day_cat', 'CompetitionMonthsOpen_cat', 'CompetitionOpenSinceYear_cat',
     'Week_cat', 'StateHoliday', 'SchoolHoliday'],
    axis=1
)

# Dynamic features - Open / Promotions / Holidays / Competition Distance
dynamic_features_open_promo_holidays_competition = df.drop(
    ['Sales', 'Date', 'Store', 'DayOfWeek', 'Year', 'Month', 'Day',
     'Promo2Weeks', 'StoreType', 'Assortment', 'PromoInterval', 'Promo2SinceYear',
     'State', 'Week', 'Events', 'Store_cat', 'Assortment_cat', 'StoreType_cat', 'State_cat',
     'SchoolHoliday_fw','SchoolHoliday_bw', 'StateHoliday_fw', 'StateHoliday_bw', 'Promo_bw', 'Promo_fw', 
     'Max_TemperatureC', 'Mean_TemperatureC', 'Min_TemperatureC', 'Max_Humidity', 'Mean_Humidity', 'Min_Humidity', 
     'Max_Wind_SpeedKm_h', 'Mean_Wind_SpeedKm_h','CloudCover', 'Events_cat',
     'trend', 'trend_DE', 'DayOfWeek_cat', 'Year_cat', 'Month_cat', 
     'Day_cat','CompetitionMonthsOpen_cat', 'CompetitionOpenSinceYear','CompetitionOpenSinceYear_cat',
     'Week_cat', 'StateHoliday', 'SchoolHoliday'],
    axis=1
)

# Dynamic features - Open / Promotions / Holidays / Competition / Weather
dynamic_features_open_promo_holidays_competition_weather = df.drop(
    ['Sales', 'Date', 'Store', 'DayOfWeek', 'Year', 'Month', 'Day',
     'Promo2Weeks', 'StoreType', 'Assortment', 'PromoInterval', 'CompetitionOpenSinceYear', 'CompetitionMonthsOpen_cat','Promo2SinceYear',
     'State', 'Week', 'Events', 'Store_cat', 'Assortment_cat', 'StoreType_cat', 'State_cat',
     'SchoolHoliday_fw','SchoolHoliday_bw', 'StateHoliday_fw', 'StateHoliday_bw', 'Promo_bw', 'Promo_fw', 
     'trend', 'trend_DE', 'DayOfWeek_cat', 'Year_cat', 'Month_cat',
     'Day_cat',
     'Week_cat', 'StateHoliday', 'SchoolHoliday'],
    axis=1
)

# Dynamic features - Open / Promotions / Holidays / Competition / Weather / Google Trend
dynamic_features_open_promo_holidays_competition_weather_googletrend = df.drop(
    ['Sales', 'Date', 'Store', 'DayOfWeek', 'Year', 'Month', 'Day' ,'CompetitionMonthsOpen_cat',
     'Promo2Weeks', 'StoreType', 'Assortment', 'PromoInterval', 'CompetitionOpenSinceYear', 'Promo2SinceYear',
     'State', 'Week', 'Events', 'Store_cat', 'Assortment_cat', 'StoreType_cat',
     'SchoolHoliday_fw','SchoolHoliday_bw', 'StateHoliday_fw', 'StateHoliday_bw', 'Promo_bw', 'Promo_fw', 
     'DayOfWeek_cat', 'Year_cat', 'Month_cat',
     'Day_cat',
     'Week_cat', 'StateHoliday', 'SchoolHoliday'],
    axis=1
)

# Display complete dynamic_features set
pd.set_option('display.max_rows', 30)
dynamic_features_open_promo_holidays_competition_weather_googletrend.head(30)

In [ ]:
# Building Dynamic Features for Rossmann

dynamic_features = dynamic_features_open_promo_holidays_competition_weather_googletrend

stack = [dynamic_features.iloc[0:number_ts,1:].values.T]
count = number_ts

for i in range(ts_length-1):
    curr_idx = count + number_ts
    stack = np.vstack((stack, [dynamic_features.iloc[count:curr_idx,1:].values.T]))
    count += number_ts
    
test_dynamic_features_list = stack.T

# Remove the last <prediction_length> days from the training set
train_dynamic_features_list = stack[:-prediction_length].T

# Validation

# Use the prediction_length days prior to the days used for 
# testing as the test set to perform hyper-parameter tuning

# training-set-length = ts_length - prediction_length*2
# testing-set-length = ts_length - prediction_length
if(validation):
    test_dynamic_features_list = stack[:-prediction_length].T
    train_dynamic_features_list = stack[:-2*prediction_length].T

# Get number of features
number_ft = len(train_dynamic_features_list[0])

# Ensure dynamic features list has the correct shape (number_ts, number_ft, ts_length)
# number_ts - number of time series | number_ft - number of features | ts_length - time series length 
print('Expected shape: (number_ts, number_features, ts_length)\n')
print(f'Number of time series: {number_ts} \nNumber of features: {number_ft}\nTime series length: {ts_length}\n')
print(f'Testing dynamic features shape: {test_dynamic_features_list.shape} \nTraining dynamic features shape (excluding last {prediction_length} time steps): {train_dynamic_features_list.shape}')


### 5.3 - Build static features list

In [ ]:
# Building Static Features for Rossmann

stores = df["Store_cat"]
assortments = df["Assortment_cat"]
store_types = df["StoreType_cat"]
states = df["State_cat"]

cat_features_cols = [stores, assortments, store_types, states]
cat_features = pd.concat(cat_features_cols, axis=1)
cat_features = cat_features.iloc[0:number_ts]

stores = cat_features["Store_cat"].values
stores_un , store_counts = np.unique(stores, return_counts=True)

assortments = cat_features["Assortment_cat"].values
assortments_un , assortments_counts = np.unique(assortments, return_counts=True)

store_types = cat_features["StoreType_cat"].values
store_types_un , store_types_counts = np.unique(store_types, return_counts=True)

states = cat_features["State_cat"].values
states_un , states_counts = np.unique(states, return_counts=True)

stat_cat_list = [stores, assortments, store_types, states]

stat_cat = np.concatenate(stat_cat_list)

stat_cat = stat_cat.reshape(len(stat_cat_list), len(stores)).T
stat_cat_cardinalities = [len(stores_un), len(assortments_un), len(store_types_un), len(states_un)]

# Ensure static categories list has the correct shape (nt, nc)
# nt - number of time series (934 stores) | number_cats - number of categories (4)
print('Expected shape: (number_ts, number_cat)\n')
print(f'Number of time series: {number_ts} \nNumber of categories: {len(stat_cat_cardinalities)}\n')
print(f'Static categories shape: {stat_cat.shape}\n')

### 5.4 - Build date list

In [ ]:
# Building date list for Rossmann

dates = df.sort_values(['Store','Date'])

dates_df = dates['Date']
dates_df = dates_df.iloc[0:ts_length]
dates = dates_df.iloc[0:ts_length].values

for i in range(len(dates)):
    dates[i] = pd.Timestamp(dates[i])
    


# Validation

# Use the prediction_length days prior to the days used for 
# testing as the test set to perform hyper-parameter tuning
if(validation):
    dates = dates[:-prediction_length]
    dates_df = dates_df[:-prediction_length]

### 5.5 - Build training and testing set

In [ ]:
# Build training and testing set from target values and both static and dynamic features for Rossmann

from gluonts.dataset.common import load_datasets, ListDataset
from gluonts.dataset.field_names import FieldName

train_ds = ListDataset([
    {
        FieldName.TARGET: target,
        FieldName.START: start,
        FieldName.FEAT_DYNAMIC_REAL: fdr,
        FieldName.FEAT_STATIC_CAT: fsc
    }
    for (target, start, fdr, fsc) in zip(train_target_values,
                                         dates,
                                         train_dynamic_features_list,
                                         stat_cat)
], freq="D")

test_ds = ListDataset([
    {
        FieldName.TARGET: target,
        FieldName.START: start,
        FieldName.FEAT_DYNAMIC_REAL: fdr,
        FieldName.FEAT_STATIC_CAT: fsc
    }
    for (target, start, fdr, fsc) in zip(test_target_values,
                                         dates,
                                         test_dynamic_features_list,
                                         stat_cat)
], freq="D")




### 5.6 - Check if sets are in the right format

In [ ]:
# This will throw an error if something is wrong, otherwise it will print
print(f'{next(iter(train_ds))}\n {next(iter(test_ds))}')

## 6 - Training Deep AR Model
#### We will be using the Negative Binomial Distribution

In [ ]:
from gluonts.model.deepar import DeepAREstimator
from gluonts.distribution.neg_binomial import NegativeBinomialOutput
from gluonts.trainer import Trainer

#deepAR_estimator = DeepAREstimator(
#    prediction_length=prediction_length,
#    context_length=2*prediction_length,
#    freq="D",
#    num_layers=5,
#    num_cells=40,
#    distr_output = NegativeBinomialOutput(),
#    use_feat_dynamic_real=True,
#    use_feat_static_cat=True,
#    cardinality=stat_cat_cardinalities,
#    dropout_rate=0.1,
#    trainer=Trainer(
#        learning_rate=1e-3,
#        epochs=15,
#        num_batches_per_epoch=500,
#        batch_size=32
#    )
#)
#

deepAR_estimator = DeepAREstimator(
    prediction_length=prediction_length,
    context_length=2*prediction_length,
    freq="D",
    num_layers=5,
    num_cells=40,
    distr_output = NegativeBinomialOutput(),
    use_feat_dynamic_real=True,
    use_feat_static_cat=True,
    cardinality=stat_cat_cardinalities,
    dropout_rate=0.1,
    trainer=Trainer(
        learning_rate=1e-3,
        epochs=20,
        num_batches_per_epoch=400,
        batch_size=32
    )
)

deepAR_predictor = deepAR_estimator.train(train_ds)




## 7. Generating forecasts

Once the estimator is fully trained, we can generate predictions from it for the test values.

In [ ]:
from gluonts.evaluation.backtest import make_evaluation_predictions

forecast_it, ts_it = make_evaluation_predictions(
    dataset=test_ds,
    predictor=deepAR_predictor,
    num_samples=100
)

print("Obtaining time series conditioning values ...")
tss = list(tqdm(ts_it, total=len(test_ds)))
print("Obtaining time series predictions ...")
forecasts = list(tqdm(forecast_it, total=len(test_ds)))

## 8. Evaluation
### 8.1 - Define evaluator

In [ ]:
from gluonts.evaluation import Evaluator

class MyEvaluator(Evaluator):

    def get_metrics_per_ts(self, time_series, forecast):
        successive_diff = np.diff(time_series.values.reshape(len(time_series)))
        successive_diff = successive_diff ** 2
        successive_diff = successive_diff[:-prediction_length]
        denom = np.mean(successive_diff)
        pred_values = forecast.samples.mean(axis=0)
        true_values = time_series.values.reshape(len(time_series))[-prediction_length:]
        num = np.mean((pred_values - true_values)**2)
        rmsse = num / denom
        metrics = super().get_metrics_per_ts(time_series, forecast)
        metrics["RMSSE"] = rmsse
        
        return metrics

    def get_aggregate_metrics(self, metric_per_ts):
        wrmsse = metric_per_ts["RMSSE"].mean()
        agg_metric , _ = super().get_aggregate_metrics(metric_per_ts)
        agg_metric["MRMSSE"] = wrmsse
        return agg_metric, metric_per_ts

evaluator = MyEvaluator(quantiles=[0.5, 0.67, 0.8, 0.95, 0.99])

### 8.2 - Generate and display metrics

In [ ]:
agg_metrics_deepAR, item_metrics_deepAR = evaluator(iter(tss), iter(forecasts), num_series=len(test_ds))

df_metrics = pd.DataFrame.from_dict(agg_metrics_deepAR, orient='index').rename(columns={0: "DeepAR"})

pd.options.display.float_format = "{:,.6f}".format

df_metrics.loc[["MASE","MAPE","sMAPE", "MSIS",  "RMSE", "NRMSE", "MRMSSE", "ND"]]
#df_metrics_2 = df_metrics.loc[['MAE', 'sMAPE', 'RMSE', 'NRMSE']]


## 9 - Plot predictions

In [ ]:
# Number of time series to plot

number_plots = 3

plot_log_path = "./plots/"
directory = os.path.dirname(plot_log_path)
if not os.path.exists(directory):
    os.makedirs(directory)
    
def plot_prob_forecasts(ts_entry, forecast_entry, path, sample_id, inline=True):
    plot_length = 200
    prediction_intervals = (50,95)
    legend = ["observations", "median prediction"] + [f"{k}% prediction interval" for k in prediction_intervals][::-1]

    _, ax = plt.subplots(1, 1, figsize=(10, 5))
    ts_entry[-plot_length:].plot(ax=ax)
    forecast_entry.plot(prediction_intervals=prediction_intervals, color='g')
    ax.axvline(ts_entry.index[-prediction_length], color='r')
    plt.legend(legend, loc="upper left")
    ax.set_ylim([-500,20000])
    if inline:
        plt.show()
        plt.clf()
    else:
        plt.savefig('{}forecast_{}.pdf'.format(path, sample_id))
        plt.close()

print("Plotting time series predictions ...")
for i in tqdm(range(number_plots)):
    ts_entry = tss[i]
    forecast_entry = forecasts[i]
    plot_prob_forecasts(ts_entry, forecast_entry, plot_log_path, i)

## 10 - Train more machine learning models for comparison
### 10.1 - Simple Feed Forward

In [ ]:
from gluonts.model.simple_feedforward import SimpleFeedForwardEstimator
from gluonts.trainer import Trainer


sff_estimator = SimpleFeedForwardEstimator(
    num_hidden_dimensions=[10],
    prediction_length=prediction_length,
    context_length=2*prediction_length,
    freq='1D',
    trainer=Trainer(
                    epochs=10,
                    learning_rate=1e-3,
                    hybridize=False,
                    num_batches_per_epoch=100
                   )
)

sff_predictor = sff_estimator.train(train_ds)

In [ ]:
forecast_it, ts_it = make_evaluation_predictions(
    dataset=test_ds,
    predictor=sff_predictor,
    num_samples=100
)

print("Obtaining time series conditioning values ...")
tss = list(tqdm(ts_it, total=len(test_ds)))
print("Obtaining time series predictions ...")
forecasts = list(tqdm(forecast_it, total=len(test_ds)))

agg_metrics_sff, item_metrics_sff = evaluator(iter(tss), iter(forecasts), num_series=len(test_ds))

df_metrics = pd.DataFrame.join(
    df_metrics,
    pd.DataFrame.from_dict(agg_metrics_sff, orient='index').rename(columns={0: "Simple Feed Forward"})
)
df_metrics.loc[["MASE","MAPE","sMAPE", "MSIS",  "RMSE", "NRMSE", "MRMSSE", "ND"]]

### 10.2 - Seasonal Naive

In [ ]:
from gluonts.model.seasonal_naive import SeasonalNaivePredictor

seasonal_predictor = SeasonalNaivePredictor(
    freq="1D", 
    prediction_length=48, 
    season_length=7)

forecast_it, ts_it = make_evaluation_predictions(test_ds, predictor=seasonal_predictor, num_samples=100)
print("Obtaining time series conditioning values ...")
tss = list(tqdm(ts_it, total=len(test_ds)))
print("Obtaining time series predictions ...")
forecasts = list(tqdm(forecast_it, total=len(test_ds)))


agg_metrics_seasonal, item_metrics_seasonal = evaluator(iter(tss), iter(forecasts), num_series=len(test_ds))

df_metrics = pd.DataFrame.join(
    df_metrics,
    pd.DataFrame.from_dict(agg_metrics_seasonal, orient='index').rename(columns={0: "Seasonal naive"})
)
df_metrics.loc[["MASE","MAPE","sMAPE", "MSIS",  "RMSE", "NRMSE", "MRMSSE", "ND"]]
#df_metrics.loc[["MAE", "sMAPE", "MSIS",  "RMSE", "NRMSE", "MRMSSE", "ND"]]

## 11 - Time series selection for single-time series models
### 11.1 - Data analysis
#### To select which time series to use in our models that only train on a single time series (so that we can compare the results with Deep AR's global model) we will perform some data analysis: 
* First we will box plot our the median of the daily sales for each store

In [ ]:
maximums = []
medians = []
averages = []
minimums = []

for i in range(len(train_target_values)):
    medians.append(np.median(train_target_values[i]))
    maximums.append(max(train_target_values[i]))
    averages.append(np.average(train_target_values[i]))
    minimums.append(min(train_target_values[i]))

In [ ]:
store_idxs = np.array(range(0,len(df.Store.unique())))
medians = np.array(medians)
df_idxs = pd.DataFrame(data=store_idxs, columns=['store'])
df_md = pd.DataFrame(data=medians, columns=['median'])
df_avg = pd.DataFrame(data=averages, columns=['average'])
df_max = pd.DataFrame(data=maximums, columns=['maximum'])
df_min = pd.DataFrame(data=minimums, columns=['minimum'])

stats = [df_idxs, df_md, df_avg, df_max, df_min]

df_stats = pd.concat(stats, axis = 1, ignore_index=True).rename(columns={0:"store_idx", 1:"median",2:"average",3:"maximum",4:"minimum"})
df_stats.head(5)




In [ ]:
import plotly.express as px

fig = px.box(df_stats, y='median',
             notched=True, # used notched shape
             title="Box plot daily sales median by store",
             hover_data=['store_idx','average','maximum','minimum'], # add store_idx column to hover data,
             points='all'
            )
fig.show()


In [ ]:
# Select 4 time series of each median quartil

# Lowest median group (between 2013 and 4677)
q1_idxs=[157, 453, 697, 701]

# Second lowest median group (between 4677 and 5868)
q2_idxs=[177,401,112, 676]

#(between 5868 and 7155)
q3_idxs=[850, 130, 384, 113]

#(between 7155 and 10647)
q4_idxs=[639, 355, 691, 360]

# Select first time serie 
#ts_target_values = test_target_values[0]

# Get target values
#ts_train_values = ts_target_values[:-prediction_length]
#ts_test_values = ts_target_values.Sales[-prediction_length:]

## 12 - Single-time series models
### 12.1 - Prophet
#### 12.1.1 - Install and import Prophet


In [ ]:
#%pip install fbprophet
from fbprophet import Prophet

#### 12.1.2 - Prepare data for Prophet
Each time series will be store in a dataframe containing the date and the target value (sales)

In [ ]:
df = df.sort_values(['Store','Date'])

ts_test_list = []
ts_train_list = []

for i in range(0,number_ts):
    curr_idx = i * ts_length
    ts = df.iloc[curr_idx:curr_idx+ts_length]
    ts = ts.drop(['StoreType', 'Assortment', 'DayOfWeek', 'Year', 'Month', 'Day' ,'CompetitionMonthsOpen_cat',
     'Promo2Weeks', 'PromoInterval', 'CompetitionOpenSinceYear', 'Promo2SinceYear',
     'State', 'Week', 'Events', 'Store_cat',
     'SchoolHoliday_fw','SchoolHoliday_bw', 'StateHoliday_fw', 'StateHoliday_bw', 'Promo_bw', 'Promo_fw', 
     'DayOfWeek_cat', 'Year_cat', 'Month_cat',
     'Day_cat','Week_cat', 'StateHoliday', 'SchoolHoliday'],axis=1)
    ts = ts.rename(columns={"Date": "ds", "Sales": "y", "trend": "gtrend"})
    ts['ds'] = pd.to_datetime(ts['ds']) # convert date to datetime
    cols = ['ds'] + ['y']  + [col for col in ts if (col != 'ds' and col !='y')]
    ts = ts[cols]
    if validation:
        ts = ts[:-prediction_length]
    ts_test = ts[-prediction_length:]
    ts_train = ts[:-prediction_length]
    ts_test_list.append(ts_test)
    ts_train_list.append(ts_train)




In [ ]:
ts_test_dates = pd.Series(dates_df, name='ds')
ts_test_dates = ts_test_dates.iloc[-prediction_length:]
ts_test_dates.reset_index(drop=True, inplace=True)

ts_train_dates = pd.Series(dates_df[:-prediction_length], name='ds')
ts_train_dates.reset_index(drop=True, inplace=True)



ts_test_list = []
ts_train_list = []

for v in test_target_values:
    
    ts_test_target = pd.Series(v[-prediction_length:])
    ts_test_target.reset_index(drop=True, inplace=True)
    
    ts_train_target = pd.Series(v[:-prediction_length])
    ts_train_target.reset_index(drop=True, inplace=True)
    
    ts_test = pd.concat([ts_test_dates, ts_test_target], axis=1)
    ts_train = pd.concat([ts_train_dates, ts_train_target], axis=1)
    
    ts_test.rename(columns = {'Date':'ds'}, inplace = True) 
    ts_test.rename(columns = {0:'y'}, inplace = True) 
    ts_train.rename(columns = {'Date':'ds'}, inplace = True) 
    ts_train.rename(columns = {0:'y'}, inplace = True) 
    ts_test_list.append(ts_test)
    ts_train_list.append(ts_train)

In [ ]:
q1_list=[]
q2_list=[]
q3_list=[]
q4_list=[]

# Create q1 list
for idx in q1_idxs:
    q1_list.append([ts_train_list[idx],ts_test_list[idx]])
# Create q2 list
for idx in q2_idxs:
    q2_list.append([ts_train_list[idx],ts_test_list[idx]])
# Create q3 list
for idx in q3_idxs:
    q3_list.append([ts_train_list[idx],ts_test_list[idx]])
# Create q4 list
for idx in q4_idxs:
    q4_list.append([ts_train_list[idx],ts_test_list[idx]])

#### 10.3.3 - Train the model, plot predictions and display error metrics

In [ ]:
# Define some common error metrics

from sklearn.metrics import mean_squared_error, mean_absolute_error
import math

def _naive_forecasting(actual: np.ndarray, seasonality: int = 1):
    return actual[:-seasonality]

def smape(actual, predicted):
    return 100/len(actual) * np.sum(2 * np.abs(predicted - actual) / (np.abs(actual) + np.abs(predicted)))

def rmse(actual: np.ndarray, predicted: np.ndarray):
    return np.sqrt(mean_squared_error(actual,predicted))

def nrmse(actual: np.ndarray, predicted: np.ndarray):
    return rmse(actual, predicted) / (actual.max() - actual.min())

def mase(actual: np.ndarray, predicted: np.ndarray, seasonality: int = 1):
    return mae(actual, predicted) / mae(actual[seasonality:], _naive_forecasting(actual, seasonality))


In [ ]:
# Create forecast list
q1_metrics = []
store_idx_idx = 0
# Train model, generate and plot forecasts for every TS in q1

for ts in q1_list:
    
    # Create model
    m = Prophet(interval_width=0.95)
    ts_train = ts[0]
    ts_test = ts[1]
    
    # Add all features
    for col in ts_train.columns:
        if(col != 'ds' and col != 'y'):
            m.add_regressor(col)
    
    # Train model
    #m.add_regressor('Open')
    m.fit(ts_train)

    pd.plotting.register_matplotlib_converters()
    
    # Get predictions
    ts_test_forecast = m.predict(ts_test)
    
    # Get metrics
    MAE = mean_absolute_error(ts_test['y'],ts_test_forecast['yhat'])
    #MASE = mase(ts_test['y'],test_forecast['yhat'])                            
    MSE = mean_squared_error(ts_test['y'],ts_test_forecast['yhat'])
    sMAPE = smape(ts_test['y'],ts_test_forecast['yhat'])
    RMSE = rmse(ts_test['y'],ts_test_forecast['yhat'])
    NRMSE = nrmse(ts_test['y'],ts_test_forecast['yhat'])
    
    # Get store idx
    store_idx = q1_idxs[store_idx_idx]
    store_idx_idx= store_idx_idx+1
    
    # Store metrics in df
    prophet_metrics = pd.DataFrame({f'Prophet_store[{store_idx}]': [MSE, sMAPE, RMSE, NRMSE]},
                      index=['MSE', 'sMAPE', 'RMSE', 'NRMSE'])
    q1_metrics.append(prophet_metrics)
    
    # Plot predictions and observed values
    pd.plotting.register_matplotlib_converters()
    
    ts_test_forecast = m.predict(ts_test)
    f, ax = plt.subplots(1)
    f.set_figheight(5)
    f.set_figwidth(15)
    ax.scatter(ts_test.ds, ts_test['y'], color='r')
    fig = m.plot(ts_test_forecast, ax=ax)
    
    # Zoom in on predicts
    f, ax = plt.subplots(figsize=(14,5))
    f.set_figheight(5)
    f.set_figwidth(15)
    ts_test.plot(kind='line',x='ds', y='y', color='red', label='Test', ax=ax)
    ts_test_forecast.plot(kind='line',x='ds',y='yhat', color='blue',label='Forecast', ax=ax)
    plt.title(f'Forecast vs Actuals in Store {store_idx}')
    plt.show()
    

In [ ]:
q1_metrics_df = pd.concat(q1_metrics, axis=1)
q1_metrics_df['mean'] = q1_metrics_df.mean(axis=1)
q1_metrics_df.head()

In [ ]:
q1_metrics_df = pd.concat(q1_metrics, axis=1)
q1_metrics_df['mean'] = q1_metrics_df.mean(axis=1)
q1_metrics_df.head()

#### Repeat the process for q2, q3 and q4 time series

In [ ]:
# Create forecast list
q2_metrics = []
store_idx_idx = 0

# Train model, generate and plot forecasts for every TS in q1
for ts in q2_list:
    
    # Train the model
    m = Prophet(interval_width=0.95)
    ts_train = ts[0]
    ts_test = ts[1]
    m.fit(ts_train)

    pd.plotting.register_matplotlib_converters()
    
    # Get predictions
    ts_test_forecast = m.predict(ts_test)
    
    # Get metrics
    MAE = mean_absolute_error(ts_test['y'],ts_test_forecast['yhat'])
    #MASE = mase(ts_test['y'],test_forecast['yhat'])                            
    MSE = mean_squared_error(ts_test['y'],ts_test_forecast['yhat'])
    sMAPE = smape(ts_test['y'],ts_test_forecast['yhat'])
    RMSE = rmse(ts_test['y'],ts_test_forecast['yhat'])
    NRMSE = nrmse(ts_test['y'],ts_test_forecast['yhat'])
    
    # Get store idx
    store_idx = q2_idxs[store_idx_idx]
    store_idx_idx= store_idx_idx+1
    
    # Store metrics in df
    prophet_metrics = pd.DataFrame({f'Prophet_store[{store_idx}]': [MSE, sMAPE, RMSE, NRMSE]},
                      index=['MSE', 'sMAPE', 'RMSE', 'NRMSE'])
    q2_metrics.append(prophet_metrics)
    
    # Plot predictions and observed values
    pd.plotting.register_matplotlib_converters()
    
    ts_test_forecast = m.predict(ts_test)
    f, ax = plt.subplots(1)
    f.set_figheight(5)
    f.set_figwidth(15)
    ax.scatter(ts_test.ds, ts_test['y'], color='r')
    fig = m.plot(ts_test_forecast, ax=ax)
    
    # Zoom in on predicts
    f, ax = plt.subplots(figsize=(14,5))
    f.set_figheight(5)
    f.set_figwidth(15)
    ts_test.plot(kind='line',x='ds', y='y', color='red', label='Test', ax=ax)
    ts_test_forecast.plot(kind='line',x='ds',y='yhat', color='blue',label='Forecast', ax=ax)
    plt.title(f'Forecast vs Actuals in Store {store_idx}')
    plt.show()
    

In [ ]:
q2_metrics_df = pd.concat(q2_metrics, axis=1)
q2_metrics_df['mean'] = q2_metrics_df.mean(axis=1)
q2_metrics_df.head()

In [ ]:
# Create forecast list
q3_metrics = []
store_idx_idx = 0

# Train model, generate and plot forecasts for every TS in q1
for ts in q3_list:
    
    # Train the model
    m = Prophet(interval_width=0.95)
    ts_train = ts[0]
    ts_test = ts[1]
    m.fit(ts_train)

    pd.plotting.register_matplotlib_converters()
    
    # Get predictions
    ts_test_forecast = m.predict(ts_test)
    
    # Get metrics
    MAE = mean_absolute_error(ts_test['y'],ts_test_forecast['yhat'])
    #MASE = mase(ts_test['y'],test_forecast['yhat'])                            
    MSE = mean_squared_error(ts_test['y'],ts_test_forecast['yhat'])
    sMAPE = smape(ts_test['y'],ts_test_forecast['yhat'])
    RMSE = rmse(ts_test['y'],ts_test_forecast['yhat'])
    NRMSE = nrmse(ts_test['y'],ts_test_forecast['yhat'])
    
    # Get store idx
    store_idx = q3_idxs[store_idx_idx]
    store_idx_idx= store_idx_idx+1
    
    # Store metrics in df
    prophet_metrics = pd.DataFrame({f'Prophet_store[{store_idx}]': [MSE, sMAPE, RMSE, NRMSE]},
                      index=['MSE', 'sMAPE', 'RMSE', 'NRMSE'])
    q3_metrics.append(prophet_metrics)
    
    # Plot predictions and observed values
    pd.plotting.register_matplotlib_converters()
    
    ts_test_forecast = m.predict(ts_test)
    f, ax = plt.subplots(1)
    f.set_figheight(5)
    f.set_figwidth(15)
    ax.scatter(ts_test.ds, ts_test['y'], color='r')
    fig = m.plot(ts_test_forecast, ax=ax)
    
    # Zoom in on predicts
    f, ax = plt.subplots(figsize=(14,5))
    f.set_figheight(5)
    f.set_figwidth(15)
    ts_test.plot(kind='line',x='ds', y='y', color='red', label='Test', ax=ax)
    ts_test_forecast.plot(kind='line',x='ds',y='yhat', color='blue',label='Forecast', ax=ax)
    plt.title(f'Forecast vs Actuals in Store {store_idx}')
    plt.show()

In [ ]:
q3_metrics_df = pd.concat(q3_metrics, axis=1)
q3_metrics_df['mean'] = q3_metrics_df.mean(axis=1)
q3_metrics_df.head()

In [ ]:
# Create forecast list
q4_metrics = []
store_idx_idx = 0

# Train model, generate and plot forecasts for every TS in q1
for ts in q4_list:
    
    # Train the model
    m = Prophet(interval_width=0.95)
    ts_train = ts[0]
    ts_test = ts[1]
    m.fit(ts_train)

    pd.plotting.register_matplotlib_converters()
    
    # Get predictions
    ts_test_forecast = m.predict(ts_test)
    
    # Get metrics
    MAE = mean_absolute_error(ts_test['y'],ts_test_forecast['yhat'])
    #MASE = mase(ts_test['y'],test_forecast['yhat'])                            
    MSE = mean_squared_error(ts_test['y'],ts_test_forecast['yhat'])
    sMAPE = smape(ts_test['y'],ts_test_forecast['yhat'])
    RMSE = rmse(ts_test['y'],ts_test_forecast['yhat'])
    NRMSE = nrmse(ts_test['y'],ts_test_forecast['yhat'])
    
    # Get store idx
    store_idx = q4_idxs[store_idx_idx]
    store_idx_idx= store_idx_idx+1
    
    # Store metrics in df
    prophet_metrics = pd.DataFrame({f'Prophet_store[{store_idx}]': [MSE, sMAPE, RMSE, NRMSE]},
                      index=['MSE', 'sMAPE', 'RMSE', 'NRMSE'])
    q4_metrics.append(prophet_metrics)
    
    # Plot predictions and observed values
    pd.plotting.register_matplotlib_converters()
    
    ts_test_forecast = m.predict(ts_test)
    f, ax = plt.subplots(1)
    f.set_figheight(5)
    f.set_figwidth(15)
    ax.scatter(ts_test.ds, ts_test['y'], color='r')
    fig = m.plot(ts_test_forecast, ax=ax)
    
    # Zoom in on predicts
    f, ax = plt.subplots(figsize=(14,5))
    f.set_figheight(5)
    f.set_figwidth(15)
    ts_test.plot(kind='line',x='ds', y='y', color='red', label='Test', ax=ax)
    ts_test_forecast.plot(kind='line',x='ds',y='yhat', color='blue',label='Forecast', ax=ax)
    plt.title(f'Forecast vs Actuals in Store {store_idx}')
    plt.show()
    

In [ ]:
q4_metrics_df = pd.concat(q4_metrics, axis=1)
q4_metrics_df['mean'] = q4_metrics_df.mean(axis=1)
q4_metrics_df.head()

#### Get average metrics for all ts

In [ ]:
average_prophet_metrics = pd.concat([q1_metrics_df.drop(columns =['mean']),
                                     q2_metrics_df.drop(columns =['mean']),
                                     q3_metrics_df.drop(columns =['mean']),
                                     q4_metrics_df.drop(columns =['mean'])], axis=1)
average_prophet_metrics['mean'] = average_prophet_metrics.mean(axis=1)
average_prophet_metrics['mean'].head()

In [ ]:
ts_train = ts_train_list[0]
ts_test = ts_test_list[0]

m = Prophet(interval_width=0.95)
m.fit(ts_train)

#### 10.3.4 - Plot predictions

In [ ]:
for forecast in q1_forecasts:
    
    pd.plotting.register_matplotlib_converters()

    ts_test_forecast = m.predict(ts_test)
    f, ax = plt.subplots(1)
    f.set_figheight(5)
    f.set_figwidth(15)
    ax.scatter(ts_test.ds, ts_test['y'], color='r')
    fig = m.plot(ts_test_forecast, ax=ax)

In [ ]:
future = m.make_future_dataframe(periods=48, freq='D')
forecast = m.predict(future)

forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

#fig = m.plot_components(forecast)

f, ax = plt.subplots(1)
f.set_figheight(5)
f.set_figwidth(15)
fig = m.plot(forecast,ax=ax)
plt.show()


#### 10.3.5 - Compare predictions with actual values

In [ ]:

pd.plotting.register_matplotlib_converters()

ts_test_forecast = m.predict(ts_test)
f, ax = plt.subplots(1)
f.set_figheight(5)
f.set_figwidth(15)
ax.scatter(ts_test.ds, ts_test['y'], color='r')
fig = m.plot(ts_test_forecast, ax=ax)


In [ ]:
f, ax = plt.subplots(figsize=(14,5))
f.set_figheight(5)
f.set_figwidth(15)
ts_test.plot(kind='line',x='ds', y='y', color='red', label='Test', ax=ax)
ts_test_forecast.plot(kind='line',x='ds',y='yhat', color='blue',label='Forecast', ax=ax)
plt.title('Forecast vs Actuals')
plt.show()

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
import math

def _naive_forecasting(actual: np.ndarray, seasonality: int = 1):
    return actual[:-seasonality]

def smape(actual, predicted):
    return 100/len(actual) * np.sum(2 * np.abs(predicted - actual) / (np.abs(actual) + np.abs(predicted)))

def rmse(actual: np.ndarray, predicted: np.ndarray):
    return np.sqrt(mean_squared_error(actual,predicted))

def nrmse(actual: np.ndarray, predicted: np.ndarray):
    return rmse(actual, predicted) / (actual.max() - actual.min())

def mase(actual: np.ndarray, predicted: np.ndarray, seasonality: int = 1):
    return mae(actual, predicted) / mae(actual[seasonality:], _naive_forecasting(actual, seasonality))

MAE = mean_absolute_error(ts_test['y'],ts_test_forecast['yhat'])

#MASE = mase(ts_test['y'],test_forecast['yhat'])                            

MSE = mean_squared_error(ts_test['y'],ts_test_forecast['yhat'])
sMAPE = smape(ts_test['y'],ts_test_forecast['yhat'])
RMSE = rmse(ts_test['y'],ts_test_forecast['yhat'])
NRMSE = nrmse(ts_test['y'],ts_test_forecast['yhat'])

prophet_metrics = pd.DataFrame({'Prophet': [MSE, sMAPE, RMSE, NRMSE]},
                  index=['MSE', 'sMAPE', 'RMSE', 'NRMSE'])

prophet_metrics.head()



In [ ]:
# Get metrics for first ts, add RMSE

# Deep AR
ts_deepAR_metrics = item_metrics_deepAR.iloc[0].to_frame()
ts_deepAR_mse = ts_deepAR_metrics.loc[['MSE']].values[0][0]
ts_deepAR_rmse = np.sqrt(ts_deepAR_mse)
deepAR_rmse_df = pd.DataFrame([[ts_deepAR_rmse]], index = ['RMSE'])
ts_deepAR_metrics = pd.concat([ts_deepAR_metrics, deepAR_rmse_df])
ts_deepAR_metrics_to_compare = ts_deepAR_metrics.loc[['sMAPE', 'RMSE']]
ts_deepAR_metrics_to_compare.columns = ['Deep AR']

# Simple Feed Forward
ts_SFF_metrics = item_metrics_sff.iloc[0].to_frame()
ts_SFF_mse = ts_SFF_metrics.loc[['MSE']].values[0][0]
ts_SFF_rmse = np.sqrt(ts_SFF_mse)
SFF_rmse_df = pd.DataFrame([[ts_SFF_rmse]], index = ['RMSE'])
ts_SFF_metrics = pd.concat([ts_SFF_metrics, SFF_rmse_df])
ts_SFF_metrics_to_compare = ts_SFF_metrics.loc[['sMAPE', 'RMSE']]
ts_SFF_metrics_to_compare.columns = ['Simple Feed Forward']

# Naive Seasonal
ts_seasonal_metrics = item_metrics_seasonal.iloc[0].to_frame()
ts_seasonal_mse = ts_seasonal_metrics.loc[['MSE']].values[0][0]
ts_seasonal_rmse = np.sqrt(ts_seasonal_mse)
seasonal_rmse_df = pd.DataFrame([[ts_seasonal_rmse]], index = ['RMSE'])
ts_seasonal_metrics = pd.concat([ts_seasonal_metrics, seasonal_rmse_df])
ts_seasonal_metrics_to_compare = ts_seasonal_metrics.loc[['sMAPE', 'RMSE']]
ts_seasonal_metrics_to_compare.columns = ['Seasonal Naive']

# Join with prophet metrics on common metrics
ts_prophet_metrics_to_compare = prophet_metrics.loc[['sMAPE', 'RMSE']]

# Show comparisons
ts_metrics = pd.concat([ts_deepAR_metrics_to_compare, ts_SFF_metrics_to_compare, ts_seasonal_metrics_to_compare, ts_prophet_metrics_to_compare], axis=1)
ts_metrics


#### 10.3.2 - Cross Validation

In [ ]:
from fbprophet.diagnostics import cross_validation
df_cv = cross_validation(m, initial='500 days', period='48 days', horizon = '48 days')
df_cv.head()

from fbprophet.diagnostics import performance_metrics
df_p = performance_metrics(df_cv)
df_p.head()

#### 10.3.3 - Hyperparameter tuning

In [ ]:
import itertools
import numpy as np
import pandas as pd

param_grid = {  
    'changepoint_prior_scale': [0.001, 0.01, 0.1, 0.5],
    'seasonality_prior_scale': [0.01, 0.1, 1.0, 10.0],
}

# Generate all combinations of parameters
all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
rmses = []  # Store the RMSEs for each params here

# Use cross validation to evaluate all parameters
for params in all_params:
    m = Prophet(**params).fit(ts)  # Fit model with given params
    df_cv = cross_validation(m, initial='500 days', period='48 days', horizon = '48 days')
    df_p = performance_metrics(df_cv, rolling_window=1)
    rmses.append(df_p['rmse'].values[0])

# Find the best parameters
tuning_results = pd.DataFrame(all_params)
tuning_results['rmse'] = rmses
print(tuning_results)

In [ ]:
best_params = all_params[np.argmin(rmses)]
print(best_params)

In [ ]:
agg_metrics_prophet, item_metrics_prophet = evaluator(iter(tss), iter(forecasts), num_series=len(test_ds))

df_metrics = pd.DataFrame.join(
    df_metrics,
    pd.DataFrame.from_dict(agg_metrics_prophet, orient='index').rename(columns={0: "Prophet"})
)
df_metrics.loc[["MASE","MAPE", "MSIS", "sMAPE", "RMSE", "NRMSE", "MRMSSE", "ND"]]

## 12. Train traditional models for comparison
* Holt Winter's Seasonal Method (additive)
* Seasonal Auto-Regressive Integrated Moving Average

### 12.1 - Holt Winter's Seasonal Method (additive)

In [ ]:
import statsmodels.api as sm
from statsmodels.tsa.api import ExponentialSmoothing

def holt_win_sea(y,y_to_train,y_to_test,seasonal_type,seasonal_period,predict_date):
    
    y.plot(marker='o', color='black', legend=True, figsize=(14, 7))
    
    if seasonal_type == 'additive':
        fit1 = ExponentialSmoothing(y_to_train, seasonal_periods = seasonal_period, trend='add', seasonal='add').fit(use_boxcox=True)
        fcast1 = fit1.forecast(predict_date).rename('Additive')
        mse1 = ((fcast1 - y_to_test) ** 2).mean()
        print('The Root Mean Squared Error of additive trend, additive seasonal of '+ 
              'period season_length={} and a Box-Cox transformation {}'.format(seasonal_period,round(np.sqrt(mse1), 2)))
        
        fit2 = ExponentialSmoothing(y_to_train, seasonal_periods = seasonal_period, trend='add', seasonal='add', damped=True).fit(use_boxcox=True)
        fcast2 = fit2.forecast(predict_date).rename('Additive+damped')
        mse2 = ((fcast2 - y_to_test) ** 2).mean()
        print('The Root Mean Squared Error of additive damped trend, additive seasonal of '+ 
              'period season_length={} and a Box-Cox transformation {}'.format(seasonal_period,round(np.sqrt(mse2), 2)))
        
        fit1.fittedvalues.plot(style='--', color='red')
        fcast1.plot(style='--', marker='o', color='red', legend=True)
        fit2.fittedvalues.plot(style='--', color='green')
        fcast2.plot(style='--', marker='o', color='green', legend=True)
    
    elif seasonal_type == 'multiplicative':  
        fit3 = ExponentialSmoothing(y_to_train, seasonal_periods = seasonal_period, trend='add', seasonal='mul').fit(use_boxcox=True)
        fcast3 = fit3.forecast(predict_date).rename('Multiplicative')
        mse3 = ((fcast3 - y_to_test) ** 2).mean()
        print('The Root Mean Squared Error of additive trend, multiplicative seasonal of '+ 
              'period season_length={} and a Box-Cox transformation {}'.format(seasonal_period,round(np.sqrt(mse3), 2)))
        
        fit4 = ExponentialSmoothing(y_to_train, seasonal_periods = seasonal_period, trend='add', seasonal='mul', damped=True).fit(use_boxcox=True)
        fcast4 = fit4.forecast(predict_date).rename('Multiplicative+damped')
        mse4 = ((fcast3 - y_to_test) ** 2).mean()
        print('The Root Mean Squared Error of additive damped trend, multiplicative seasonal of '+ 
              'period season_length={} and a Box-Cox transformation {}'.format(seasonal_period,round(np.sqrt(mse4), 2)))
        
        fit3.fittedvalues.plot(style='--', color='red')
        fcast3.plot(style='--', marker='o', color='red', legend=True)
        fit4.fittedvalues.plot(style='--', color='green')
        fcast4.plot(style='--', marker='o', color='green', legend=True)
        
    else:
        print('Wrong Seasonal Type. Please choose between additive and multiplicative')

    plt.show()

seasonality_frequency = 28
    
holt_win_sea(ts_target_values, ts_train_values, ts_test_values,'additive',52, prediction_length)

### 12.2 - Season Auto-Regressive Integrated Moving Average (SARIMA)
#### In order to get the best prediction, it’s important to find the values of SARIMA(p,d,q)(P,D,Q)m that optimize a metric of interest. We will use a "grid search" to iteratively explore different combinations of parameters. 

In [ ]:
import itertools

def sarima_grid_search(y,seasonal_period):
    p = d = q = range(0, 2)
    pdq = list(itertools.product(p, d, q))
    seasonal_pdq = [(x[0], x[1], x[2],seasonal_period) for x in list(itertools.product(p, d, q))]
    
    mini = float('+inf')
    
    
    for param in pdq:
        for param_seasonal in seasonal_pdq:
            try:
                mod = sm.tsa.statespace.SARIMAX(y,
                                                order=param,
                                                seasonal_order=param_seasonal,
                                                enforce_stationarity=False,
                                                enforce_invertibility=False)

                results = mod.fit()
                
                if results.aic < mini:
                    mini = results.aic
                    param_mini = param
                    param_seasonal_mini = param_seasonal

#                 print('SARIMA{}x{} - AIC:{}'.format(param, param_seasonal, results.aic))
            except:
                continue
    print('The set of parameters with the minimum AIC is: SARIMA{}x{} - AIC:{}'.format(param_mini, param_seasonal_mini, mini))
    
sarima_grid_search(ts_target_values,seasonality_frequency)

In [ ]:
# Call this function after pick the right(p,d,q) for SARIMA based on AIC               
def sarima_eva(y,order,seasonal_order,seasonal_period,pred_date,y_to_test):
    # fit the model 
    mod = sm.tsa.statespace.SARIMAX(y,
                                order=order,
                                seasonal_order=seasonal_order,
                                enforce_stationarity=False,
                                enforce_invertibility=False)

    results = mod.fit()
    print(results.summary().tables[1])
    
    results.plot_diagnostics(figsize=(16, 8))
    plt.show()
    
    # The dynamic=False argument ensures that we produce one-step ahead forecasts, 
    # meaning that forecasts at each point are generated using the full history up to that point.
    pred = results.get_prediction(start=pd.to_datetime(pred_date), dynamic=False)
    pred_ci = pred.conf_int()
    y_forecasted = pred.predicted_mean
    mse = ((y_forecasted - y_to_test) ** 2).mean()
    print('The Root Mean Squared Error of SARIMA with season_length={} and dynamic = False {}'.format(seasonal_period,round(np.sqrt(mse), 2)))

    ax = y.plot(label='observed')
    y_forecasted.plot(ax=ax, label='One-step ahead Forecast', alpha=.7, figsize=(14, 7))
    ax.fill_between(pred_ci.index,
                    pred_ci.iloc[:, 0],
                    pred_ci.iloc[:, 1], color='k', alpha=.2)

    ax.set_xlabel('Date')
    ax.set_ylabel('Sessions')
    plt.legend()
    plt.show()

    # A better representation of our true predictive power can be obtained using dynamic forecasts. 
    # In this case, we only use information from the time series up to a certain point, 
    # and after that, forecasts are generated using values from previous forecasted time points.
    pred_dynamic = results.get_prediction(start=pd.to_datetime(pred_date), dynamic=True, full_results=True)
    pred_dynamic_ci = pred_dynamic.conf_int()
    y_forecasted_dynamic = pred_dynamic.predicted_mean
    mse_dynamic = ((y_forecasted_dynamic - y_to_test) ** 2).mean()
    print('The Root Mean Squared Error of SARIMA with season_length={} and dynamic = True {}'.format(seasonal_period,round(np.sqrt(mse_dynamic), 2)))

    ax = y.plot(label='observed')
    y_forecasted_dynamic.plot(label='Dynamic Forecast', ax=ax,figsize=(14, 7))
    ax.fill_between(pred_dynamic_ci.index,
                    pred_dynamic_ci.iloc[:, 0],
                    pred_dynamic_ci.iloc[:, 1], color='k', alpha=.2)

    ax.set_xlabel('Date')
    ax.set_ylabel('Sessions')

    plt.legend()
    plt.show()
    
    return (results)

sarima_eva(ts_target_values,(1, 1, 1),(1, 1, 0, seasonality_frequency),seasonality_frequency,'2019-06-02',ts_test_values)